# Cox, Ross and Rubinstein Binomial Method for Options Pricing
> Python Dynamic Programming and Numpy implementations

In [8]:
import math
import numpy as np
import yfinance as yf
from time import time
from functools import wraps

In [9]:
def calc_runtime(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        print(f"{func.__name__}({args, kwargs}) executed in ", end="")
        print(f"{end_time - start_time:.6f} seconds")
        return result
    return wrapper

# Initialize input data

In [ ]:
S0 = 120.0     # Initial Stock Price
SK = 430.0     # Strike Price
r = 0.055      # Risk-free interest rate
T = 6          # Time to maturity in years
N = 3000          # Number of time steps
opttype = "C"  # "C" for Call, "P" for Put  


# CRR values
### Calculating the up (up) and down (d) values using volatility (σ)

## CRR using infered σ from u and d

In [25]:
# u and d inferered from CRR model

## CRR using implied σ
> found by reverse calculating σ from option market price

In [26]:
# connect too Yahoo Finance to get stock/option data

# Algorithm / Python For Loop Implementation

In [23]:
import numpy as np

@calc_runtime
def binomial_tree_loop(
    S0: float,
    SK: float,
    r: float,
    T: float,
    N: int,
    u: float,
    d: float,
    opttype: str = "C",
    american: bool = False,
    debug: bool = False,
) -> float:
    """
    Binomial tree pricer using 1D arrays (implicit tree).
    - opttype: "C" (call) or "P" (put)
    - american: if True, allows early exercise via max(continuation, intrinsic)
    - debug: if True, prints the in/out row values each step
    """

    if N <= 0:
        raise ValueError("N must be a positive integer.")
    if u == d:
        raise ValueError("u and d must be different.")
    if d <= 0 or u <= 0:
        raise ValueError("u and d must be positive.")
    opt = opttype.upper()
    if opt not in ("C", "P"):
        raise ValueError("opttype must be 'C' or 'P'.")

    dt = T / N
    disc = np.exp(-r * dt)
    q = (np.exp(r * dt) - d) / (u - d)

    if not (0.0 < q < 1.0):
        # Not fatal for code execution, but usually indicates inconsistent parameters
        # (violates no-arbitrage condition d < exp(r*dt) < u)
        if debug:
            print(f"WARNING: q={q:.6f} not in (0,1). Check: d < exp(r*dt) < u")

    def payoff(s: float) -> float:
        if opt == "C":
            return max(0.0, s - SK)
        else:  # "P"
            return max(0.0, SK - s)

    # --- Build terminal stock prices S_N(j), j=0..N (the last row of the tree) ---
    S = np.zeros(N + 1, dtype=float)
    S[0] = S0 * (d ** N)                   # all-down terminal node
    step = (u / d)
    for j in range(1, N + 1):
        S[j] = S[j - 1] * step             # move across terminal layer

    # --- Terminal option values ---
    C = np.array([payoff(s) for s in S], dtype=float)

    if debug:
        print("\n=== Terminal layer (time N) ===")
        print("S_N:", S)
        print("C_N:", C)
        print(f"dt={dt:.6f}, q={q:.6f}, disc={disc:.6f}, american={american}, opt={opt}")

    # --- Backward induction: collapse row i -> i-1 in-place ---
    for i in range(N, 0, -1):
        if debug:
            print(f"\n--- Collapse time {i} -> {i-1} ---")
            print(f"Input C_i (len {i+1}):", C[: i + 1])
            if american:
                print(f"Input S_i (len {i+1}):", S[: i + 1])

        for j in range(i):
            down_val = C[j]      # C_i(j)
            up_val = C[j + 1]    # C_i(j+1)

            cont = disc * (q * up_val + (1.0 - q) * down_val)

            if not american:
                C[j] = cont
                if debug:
                    print(
                        f" node (i-1={i-1}, j={j}): "
                        f"down={down_val:.6f}, up={up_val:.6f} -> cont={cont:.6f}"
                    )
            else:
                # Roll stock prices back one step for this node: S_{i-1}(j) = S_i(j)/d
                S[j] = S[j] / d
                exercise = payoff(S[j])
                C[j] = max(cont, exercise)

                if debug:
                    print(
                        f" node (i-1={i-1}, j={j}): "
                        f"down={down_val:.6f}, up={up_val:.6f} -> cont={cont:.6f} | "
                        f"S_{i-1}={S[j]:.6f}, exercise={exercise:.6f} -> new={C[j]:.6f}"
                    )

        if debug:
            print(f"Output C_{i-1} (len {i}):", C[:i])
            if american:
                print(f"Output S_{i-1} (len {i}):", S[:i])

    if debug:
        print("\n=== Price at time 0 ===")
        print("C_0:", C[0])

    return float(C[0])


# Algorithm / NumPy Vectorized Implementation

# Algorithm / C++ Implementation (Using Python Bindings)

# Testing

In [24]:
binomial_tree_loop(S0, SK, r, T, N, u, d, opttype)

binomial_tree_loop(((120.0, 430.0, 0.055, 6, 3000, 1.7, 0.5882352941176471, 'C'), {})) executed in 1.207270 seconds


0.0